# 4. Interprocess Communication

UDP Datagrams
- Connection-less
- Barebones

TCP Streams
- Connection-oriented
- Reliable-ish (Validity, Integrity, Ordering)

Synchronous Communication
- after `send` a process is blocked until a corresponding `receive` at the recipient process.
- after `receive` a process is blocked until a message arrives.

Asynchronous
- nonblocking `send` 
- blocking or nonblocking `receive`; in case of non-blocking need to manage buffer for when message arrives as well as notification of arrival - nonblocking receive is awkward and generally not provided

Address
*(IP address, port)* 
Ports can have a single receiver but many senders

Reliability
*guaranteed delivery despite a reasonable number of lost packets*
- Arrives within reasonable expectations
- Uncorrupted
- No duplications
(May or may not be in sender-order, TCP does this though)

## Tools

`nslookup` - DNS Lookup

`nmap` - Port scanning

`nc` - Communication

`traceroute` - Route tracer

`ifconfig` - Network configuration

`tcpdump` - Dump traffic from network

`ping` - duh

## Sockets (In Java)

## `InetAddress`

In [21]:
import java.net.InetAddress;
InetAddress addr = InetAddress.getByName("google.com");

System.out.println(addr);

google.com/216.58.207.238


In [37]:
import java.net.UnknownHostException;
Function<String,InetAddress> getByName = (String name) -> { try {
    return InetAddress.getByName(name);
} catch (UnknownHostException e) { return null; } }


In [58]:
System.out.println(getByName.apply("0.0.0.0").isAnyLocalAddress());
System.out.println(getByName.apply("00:00:00:00:00:00:00:1").getHostName());
System.out.println(getByName.apply("google.com"));

true
localhost
google.com/142.250.74.46


## UDP

In [20]:
import java.net.*;

try ( 
    DatagramSocket server = new DatagramSocket(8888);
    DatagramSocket client = new DatagramSocket();
) {
    // Retrieve local internet address (127.0.0.1/)
    InetAddress addr = InetAddress.getByName("localhost");
    // Allocate I/O buffers for traffic
    byte[] serverbuff = new byte[1024];
    byte[] clientbuff = "Hello, Senor Mundo!".getBytes();
    
    // Issue a send, moving the data from the buffer to the underlying layer
    DatagramPacket pack = new DatagramPacket(clientbuff, clientbuff.length, addr, 8888);
    client.send(pack);
  
    // Receive any message from the underlying incoming buffer
    pack = new DatagramPacket(serverbuff, 0, 1024);
    server.receive(pack);
    System.out.println(new String(pack.getData()));
    System.out.println("SO_SNDBUF = " + server.getSendBufferSize());
    System.out.println("SO_RCVBUF = " + server.getReceiveBufferSize());
    System.out.println("SO_REUSEADDR = " + server.getReuseAddress());
    System.out.println("IP_TOS = " + server.getTrafficClass());
    System.out.println("SO_BROADCAST = " + server.getBroadcast());
} catch (Exception e) {
    e.printStackTrace();
}

Hello, Senor Mundo!                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                     

`receive` will block until a message is received from any source, it is possible to connect the socket to a remote addr/port such that messages are exchanged only with that address. If a socket is not connected to any such remote, we say that it is *unconnected*

socket options should be set before binding the socket! 
see [SocketOptions](https://docs.oracle.com/en/java/javase/15/docs/api/java.base/java/net/StandardSocketOptions.html#SO_REUSEADDR)

UDP uses checksums to supply integrity, but suffers from send/receive-omission failures as well as ordering failures, and byzantine failures (e.g. channel-omission or malicious interference).

## TCP

In [35]:
// BE HELLA CAREFUL WITH CLOSING STREAMS/SOCKETS!!!!!
// Charsets included for locale independence

import java.io.*;
import java.nio.charset.*;

Charset utf8 = StandardCharsets.UTF_8;

try (
    ServerSocket server = new ServerSocket(9999);
    Socket client = new Socket("localhost", 9999);
    Socket clientHandle = server.accept(); // blocking
    
    PrintWriter servW = 
        new PrintWriter(clientHandle.getOutputStream(), true, utf8);
    BufferedReader servR = 
        new BufferedReader(new InputStreamReader(clientHandle.getInputStream(), utf8));
    PrintWriter clieW = 
        new PrintWriter(client.getOutputStream(), true, utf8);
    BufferedReader clieR = 
        new BufferedReader(new InputStreamReader(client.getInputStream(), utf8));
) {
    servW.println("Server says hi!");
    System.out.println(clieR.readLine());
    clieW.println("Client says hi too!");
    System.out.println(servR.readLine());
} catch (Exception e) {
    e.printStackTrace();
}

Server says hi!
Client says hi too!


## Marshalling

*Little Endian order* :  most significant byte last

*Big Endian order* : most significant byte first

*External data representation* : external agreed upon representation of data

*Marshalling* : the process of transforming data items into a form suitable for transmission. We call the inverse process *Unmarshalling*, here data structures and local representations are reestablished.

Approaches to marshalling : 
- Convention-based (e.g. CORBA)
- Built-in standard (java serialization)
- Self-contained data representation (XML)

### Java Serialization

Objects are serialized recursively!!

Uses `ObjectOutputStream` and `ObjectInputStream` objects to R/W objects.

`transient` keyword can be used to omit fields from serialized form.

by implementing `readObject(ObjectInputStream in)` and/or `writeObject(ObjectOutputStream out)` the serialization can be handled in more detail.

In [16]:
import java.io.*;

class Person implements Serializable {
    String name;
    int age;
    transient long cpr;
    
    Person(String name, int age, long cpr) {
        this.name = name;
        this.age = age;
        this.cpr = cpr;
    }

    public String toString() {
        return name + " " + age + " " + cpr;
    }

    private void writeObject(ObjectOutputStream oos) throws IOException {
        oos.defaultWriteObject();
        oos.writeLong(~cpr); // weakest encryption world
    }
    
    private void readObject(ObjectInputStream ois) throws ClassNotFoundException, IOException {
        ois.defaultReadObject();
        cpr = ~ois.readLong();
    }
}

In [17]:
import java.io.*;

Person Hans = new Person("Hans", 88, 24032001);

try (
    PipedInputStream in = new PipedInputStream();
    PipedOutputStream out = new PipedOutputStream(in); 
) {
    ObjectOutputStream oos = new ObjectOutputStream(out);
    ObjectInputStream ois = new ObjectInputStream(in);
    
    oos.writeObject(Hans);
    System.out.println(ois.readObject());
} catch (Exception e) {
    e.printStackTrace();
}

Hans 88 24032001


## XML

`<i am a tag></i am a tag>`

`<i am an empty tag />`

*elements* are enclosed by *tags*

*attributes* hold information about an element

<tag attributekey="attributevalue" />

use e.g. base64 for raw data

use CDATA to pass strings verbatim (CDATA will not be parsed as XML)

A *prolog* is included to specify encoding, xml version and more.

`<?XML version = "1.0" encoding = "UTF-8" standalone = "yes"?>`

XML has a feature for namespacing elements (by url for example)

## Multicast

Allows for joining/leaving multicast groups, everything else is mostly just UDP.